In [21]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib


In [22]:
data = pd.read_csv('/home/tsoien/github/MLResearch/backend/data/Disease_symptom_and_patient_profile_dataset.csv')
df = pd.DataFrame(data)

# Display the first few rows
print(df.head())

# Check dataset information
print(df.info())


       Disease Fever Cough Fatigue Difficulty Breathing  Age  Gender  \
0    Influenza   Yes    No     Yes                  Yes   19  Female   
1  Common Cold    No   Yes     Yes                   No   25  Female   
2       Eczema    No   Yes     Yes                   No   25  Female   
3       Asthma   Yes   Yes      No                  Yes   25    Male   
4       Asthma   Yes   Yes      No                  Yes   25    Male   

  Blood Pressure Cholesterol Level Outcome Variable  
0            Low            Normal         Positive  
1         Normal            Normal         Negative  
2         Normal            Normal         Negative  
3         Normal            Normal         Positive  
4         Normal            Normal         Positive  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349 entries, 0 to 348
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Disease               349 non-nul

In [23]:
# Define features (X) and target variable (y)
target_column = 'Outcome Variable'
X = df.drop(columns=[target_column])  # Drop the target column to keep features
y = df[target_column]  # Target column


In [24]:
from sklearn.impute import SimpleImputer

# Identify categorical and numeric columns
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Create preprocessing pipelines
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Combine into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# Preprocess the data
X_preprocessed = preprocessor.fit_transform(X)

# Check the shape of the preprocessed data
print("Preprocessed Data Shape:", X_preprocessed.shape)


Preprocessed Data Shape: (349, 133)


In [25]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Check the sizes of splits
print("Training Set Size:", X_train.shape)
print("Testing Set Size:", X_test.shape)


Training Set Size: (279, 133)
Testing Set Size: (70, 133)


In [27]:
# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Save the trained model
model_path = '/home/tsoien/github/MLResearch/backend/ml_model/model.joblib'
joblib.dump(rf_model, model_path)
print(f"Model saved to {model_path}")


Model saved to /home/tsoien/github/MLResearch/backend/ml_model/model.joblib


In [28]:
# Make predictions
y_pred = rf_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.81
Classification Report:
              precision    recall  f1-score   support

    Negative       0.74      0.87      0.80        30
    Positive       0.89      0.78      0.83        40

    accuracy                           0.81        70
   macro avg       0.81      0.82      0.81        70
weighted avg       0.82      0.81      0.82        70

